# __311 Service Requests in Chicago__

## Import dependencies

In [0]:
!pip install altair -U
!pip install geopandas -U
!sudo apt-get update && apt-get install -y libspatialindex-dev
!pip install rtree
from google.colab import drive
drive.mount('/content/gdrive')
import altair as alt
import pandas as pd
import geopandas as gpd

     |████████████████████████████████| 716kB 8.6MB/s 
  Found existing installation: altair 4.0.0
    Uninstalling altair-4.0.0:
      Successfully uninstalled altair-4.0.0
     |████████████████████████████████| 921kB 9.5MB/s 
     |████████████████████████████████| 10.1MB 38.3MB/s 
     |████████████████████████████████| 11.8MB 24.0MB/s 
Get:1 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:4 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease [15.4 kB]
Get:5 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:6 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:9 https://developer.down

## Helper Functions

In [0]:
def show_col_row_count(df):

    row, col = df.shape
    msg = "The dataframe currently has {} rows and {} columns."
    print(msg.format(row, col))

def drop_non_numeric(df, columns, verbose=False):

    if verbose:
        rows, _ = df.shape
        col_msg = "Removing all non-numeric rows for the following columns: {}."
        print(col_msg.format(columns))
    for col in columns:
        df = df[pd.to_numeric(df[col], errors="coerce").notnull()]
        if verbose:
            new_rows, _ = df.shape
            print("Removed {} rows from {}.".format(rows - new_rows, col))
            rows = new_rows
    return df

def drop_na(dataframe, columns, verbose=False):

    if verbose:
        rows, _ = dataframe.shape
        col_msg = "Removing all rows with NAs for the following columns: {}."
        print(col_msg.format(columns))
    df = dataframe.dropna(subset = columns)
    if verbose:
        new_rows, _ = dataframe.shape
        print("Removed {} rows.".format(rows - new_rows))
    return df

def save_columns(dataframe, columns):

    dataframe = dataframe[columns]
    return dataframe

def df_to_csv(dataframe, filename, verbose=False):

    try:
        dataframe.to_csv(filename, index=False)
        if verbose:
            print("Successfully saved to {}.".format(filename))
    except:
        if verbose:
            print("Failed to save to {}.".format(filename))

## Load the dataset - `311_Service_Requests.csv`

In [0]:
data_dir = "/content/gdrive/My Drive/CAPP-30239-Data-Viz/assignment-2/data/311_Service_Requests_Chicago.csv"
dtype_dict = {
    "STREET_NUMBER": "object",
    "LEGACY_SR_NUMBER": "object",
    "PARENT_SR_NUMBER": "object",
    "SANITATION_DIVISION_DAYS": "object",
    }
df = pd.read_csv(data_dir, dtype=dtype_dict)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:
df["CREATED_DATE"] = pd.to_datetime(df["CREATED_DATE"], format='%m/%d/%Y %I:%M:%S %p')
df["CREATED_YEAR"] = df["CREATED_DATE"].dt.year
df["CLOSED_DATE"].fillna("01/16/2020 10:20:00 PM", inplace = True) 
df["CLOSED_DATE"]= pd.to_datetime(df["CLOSED_DATE"], format='%m/%d/%Y %I:%M:%S %p')
df["CLOSED_YEAR"] = df["CLOSED_DATE"].dt.year
df["DAYS_TAKEN"] = df["CLOSED_DATE"] - df["CREATED_DATE"]
df["DAYS_TAKEN"] = df["DAYS_TAKEN"].dt.days

In [0]:
df.sample(n = 5) 

,SR_NUMBER,SR_TYPE,SR_SHORT_CODE,OWNER_DEPARTMENT,STATUS,CREATED_DATE,LAST_MODIFIED_DATE,CLOSED_DATE,STREET_ADDRESS,CITY,STATE,ZIP_CODE,STREET_NUMBER,STREET_DIRECTION,STREET_NAME,STREET_TYPE,DUPLICATE,LEGACY_RECORD,LEGACY_SR_NUMBER,PARENT_SR_NUMBER,COMMUNITY_AREA,WARD,ELECTRICAL_DISTRICT,ELECTRICITY_GRID,POLICE_SECTOR,POLICE_DISTRICT,POLICE_BEAT,PRECINCT,SANITATION_DIVISION_DAYS,CREATED_HOUR,CREATED_DAY_OF_WEEK,CREATED_MONTH,X_COORDINATE,Y_COORDINATE,LATITUDE,LONGITUDE,LOCATION,CREATED_YEAR,CLOSED_YEAR,DAYS_TAKEN
225400,SR19-01795040,Street Light Out Complaint,SFD,CDOT - Department of Transportation,Completed,2019-06-25 22:14:16,06/28/2019 04:03:19 PM,2019-06-28 16:03:19,6952 S HAMLIN AVE,NaN,NaN,60629,6952,S,HAMLIN,AVE,True,False,NaN,SR19-01767903,65.0,13.0,NaN,G038,3.0,8.0,833.0,41.0,NaN,22,3,6,1.152230e+06,1.858011e+06,41.766252,-87.717589,"(41.76625182616667, -87.71758903796865)",2019,2019,2
1398646,SR19-00442316,Tree Debris Clean-Up Request,SEL,Streets and Sanitation,Completed,2018-09-10 15:24:13,05/17/2019 11:08:17 AM,2018-09-14 12:06:40,6610 S WINCHESTER AVE,CHICAGO,IL,60636,6610,S,WINCHESTER,AVE,False,True,CSR18-02626450,NaN,67.0,15.0,11.0,L037,2.0,7.0,726.0,7.0,Friday,15,2,9,1.164487e+06,1.860759e+06,41.773542,-87.672586,"(41.77354165680971, -87.67258613217898)",2018,2018,3
610088,SR19-02477583,Report an Injured Animal,EAQ,Animal Care and Control,Completed,2019-09-13 13:53:09,09/14/2019 12:21:34 PM,2019-09-14 12:21:34,4825 W FLETCHER ST,NaN,NaN,60641,4825,W,FLETCHER,ST,False,False,NaN,NaN,19.0,31.0,NaN,D015,2.0,25.0,2521.0,9.0,NaN,13,6,9,1.143640e+06,1.920478e+06,41.937833,-87.747513,"(41.93783279630803, -87.74751332959913)",2019,2019,0
1690000,SR19-01041892,311 INFORMATION ONLY CALL,311IOC,311 City Services,Completed,2019-02-20 10:18:03,05/17/2019 03:43:25 PM,2019-02-20 10:18:03,2111 W Lexington ST,NaN,NaN,60612,2111,W,Lexington,ST,False,False,NaN,NaN,28.0,28.0,7.0,K024,2.0,12.0,1225.0,46.0,Friday,10,4,2,1.162234e+06,1.896560e+06,41.871831,-87.679846,"(41.87183127800069, -87.67984622001485)",2019,2019,0
1854843,SR19-01289056,Aircraft Noise Complaint,AVN,Aviation,Completed,2019-04-07 23:15:53,05/17/2019 04:16:49 PM,2019-04-07 23:15:53,10510 W ZEMKE RD,Chicago,IL,60018,10510,W,ZEMKE,RD,False,False,NaN,NaN,76.0,41.0,NaN,NaN,5.0,16.0,1654.0,27.0,NaN,23,1,4,1.105376e+06,1.941043e+06,41.994897,-87.887752,"(41.99489654897978, -87.88775165537639)",2019,2019,0


In [0]:
show_col_row_count(df)

The dataframe currently has 2362297 rows and 40 columns.


In [0]:
df = drop_non_numeric(df, ["ZIP_CODE"], verbose=True)

Removing all non-numeric rows for the following columns: ['ZIP_CODE'].
Removed 17961 rows from ZIP_CODE.


In [0]:
df = drop_na(df, ["ZIP_CODE", "CREATED_DATE", "WARD", "LOCATION", "LATITUDE", "LATITUDE"], verbose=True)

Removing all rows with NAs for the following columns: ['ZIP_CODE', 'CREATED_DATE', 'WARD', 'LOCATION', 'LATITUDE', 'LATITUDE'].
Removed 0 rows.


In [0]:
columns = [
    "SR_NUMBER", "SR_TYPE", "SR_SHORT_CODE",
    "OWNER_DEPARTMENT","STATUS", "CREATED_DATE",
    "CLOSED_DATE", "ZIP_CODE", "DUPLICATE",
    "LEGACY_RECORD", "WARD", "LATITUDE",
    "LONGITUDE", "LOCATION",
    "DAYS_TAKEN", "CREATED_YEAR", "CLOSED_YEAR",
    "CREATED_HOUR",	"CREATED_DAY_OF_WEEK", "CREATED_MONTH"
]
df = save_columns(df, columns)

In [0]:
df.head(n = 5) 

,SR_NUMBER,SR_TYPE,SR_SHORT_CODE,OWNER_DEPARTMENT,STATUS,CREATED_DATE,CLOSED_DATE,ZIP_CODE,DUPLICATE,LEGACY_RECORD,WARD,LATITUDE,LONGITUDE,LOCATION,DAYS_TAKEN,CREATED_YEAR,CLOSED_YEAR,CREATED_HOUR,CREATED_DAY_OF_WEEK,CREATED_MONTH
11,SR19-02763102,Sewer Cave-In Inspection Request,AAD,DWM - Department of Water Management,Completed,2019-10-19 15:31:55,2019-11-25 13:34:43,60625,False,False,47.0,41.966001,-87.687281,"(41.96600085415316, -87.6872811861682)",36,2019,2019,15,7,10
171,SR19-02366062,311 INFORMATION ONLY CALL,311IOC,311 City Services,Completed,2019-09-02 07:55:43,2019-09-02 07:55:43,60612,False,False,28.0,41.871831,-87.679846,"(41.871831277993564, -87.67984621876099)",0,2019,2019,7,2,9
175,SR19-02366112,Aircraft Noise Complaint,AVN,Aviation,Completed,2019-09-02 08:11:53,2019-09-02 08:11:53,60666,False,False,41.0,41.994897,-87.887752,"(41.9948965489749, -87.8877516541201)",0,2019,2019,8,2,9
205,SR19-01480551,Pothole in Street Complaint,PHF,CDOT - Department of Transportation,Completed,2019-05-13 12:59:56,2019-11-02 08:01:21,60619,True,False,8.0,41.742149,-87.604865,"(41.74214930362783, -87.60486531173716)",172,2019,2019,12,2,5
586,SR19-02763151,311 INFORMATION ONLY CALL,311IOC,311 City Services,Completed,2019-10-19 15:42:07,2019-10-19 15:42:07,60612,False,False,38.0,41.953736,-87.768543,"(41.95373571422855, -87.76854307874956)",0,2019,2019,15,7,10


In [0]:
show_col_row_count(df)

The dataframe currently has 2325604 rows and 20 columns.


In [0]:
# Save the dataset

# Dataset for 2018/07/01 - 2018/12/31
df_2018 = df[(df['CREATED_DATE'] > '2018-07-01 00:00:00') & (df['CREATED_DATE'] <= '2018-12-31 23:59:59')]
filename_2018 = "/content/gdrive/My Drive/CAPP-30239-Data-Viz/assignment-2/data/cleaned_311_2018.csv"
df_to_csv(df_2018, filename_2018, verbose=True)
# Dataset for 2019/07/01 - 2019/12/31
df_2019 = df[(df['CREATED_DATE'] > '2019-07-01 00:00:00') & (df['CREATED_DATE'] <= '2019-12-31 23:59:59')]
filename_2019 = "/content/gdrive/My Drive/CAPP-30239-Data-Viz/assignment-2/data/cleaned_311_2019.csv"
df_to_csv(df_2019, filename_2019, verbose=True)
# Entire cleaned dataset
filename = "/content/gdrive/My Drive/CAPP-30239-Data-Viz/assignment-2/data/cleaned_311.csv"
df_to_csv(df, filename, verbose=True)

Successfully saved to /content/gdrive/My Drive/CAPP-30239-Data-Viz/assignment-2/data/cleaned_311_2018.csv.
Successfully saved to /content/gdrive/My Drive/CAPP-30239-Data-Viz/assignment-2/data/cleaned_311_2019.csv.
Successfully saved to /content/gdrive/My Drive/CAPP-30239-Data-Viz/assignment-2/data/cleaned_311.csv.


## Create a dataset for Census Tract (Number of Requests and Income Level)



In [0]:
# Load Census Tract Polygon dataset
tract_dir = "/content/gdrive/My Drive/CAPP-30239-Data-Viz/assignment-2/data/census_tract_boundary/tracts.geojson"
tracts = gpd.read_file(tract_dir)
tracts.crs = 'epsg:4326'
tracts.head()

,statefp10,name10,commarea_n,namelsad10,commarea,geoid10,notes,tractce10,countyfp10,geometry
0,17,8424,44,Census Tract 8424,44,17031842400,,842400,031,"MULTIPOLYGON (((-87.62405 41.73022, -87.62405 ..."
1,17,8403,59,Census Tract 8403,59,17031840300,,840300,031,"MULTIPOLYGON (((-87.68608 41.82296, -87.68607 ..."
2,17,8411,34,Census Tract 8411,34,17031841100,,841100,031,"MULTIPOLYGON (((-87.62935 41.85280, -87.62934 ..."
3,17,8412,31,Census Tract 8412,31,17031841200,,841200,031,"MULTIPOLYGON (((-87.68813 41.85569, -87.68816 ..."
4,17,8390,32,Census Tract 8390,32,17031839000,,839000,031,"MULTIPOLYGON (((-87.63312 41.87449, -87.63306 ..."


In [0]:
cleaned_311 = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.LONGITUDE, df.LATITUDE))

In [0]:
cleaned_311.sample(n=1)

,SR_NUMBER,SR_TYPE,SR_SHORT_CODE,OWNER_DEPARTMENT,STATUS,CREATED_DATE,CLOSED_DATE,ZIP_CODE,DUPLICATE,LEGACY_RECORD,WARD,LATITUDE,LONGITUDE,LOCATION,DAYS_TAKEN,CREATED_YEAR,CLOSED_YEAR,CREATED_HOUR,CREATED_DAY_OF_WEEK,CREATED_MONTH,geometry
96006,SR19-01674218,Aircraft Noise Complaint,AVN,Aviation,Completed,2019-06-10 16:49:37,2019-06-10 16:49:38,60666,False,False,41.0,41.994897,-87.887752,"(41.99489654897978, -87.88775165537639)",0,2019,2019,16,2,6,POINT (-87.88775 41.99490)


In [0]:
cleaned_311.crs = "epsg:4326"
cleaned_311 = cleaned_311[cleaned_311.geometry.type == 'Point']
tracts_311 = gpd.sjoin(cleaned_311, tracts, how="inner", op='intersects')

In [0]:
tracts_311.sample(n=1)

,SR_NUMBER,SR_TYPE,SR_SHORT_CODE,OWNER_DEPARTMENT,STATUS,CREATED_DATE,CLOSED_DATE,ZIP_CODE,DUPLICATE,LEGACY_RECORD,WARD,LATITUDE,LONGITUDE,LOCATION,DAYS_TAKEN,CREATED_YEAR,CLOSED_YEAR,CREATED_HOUR,CREATED_DAY_OF_WEEK,CREATED_MONTH,geometry,index_right,statefp10,name10,commarea_n,namelsad10,commarea,geoid10,notes,tractce10,countyfp10
1515394,SR19-00656211,Abandoned Vehicle Complaint,SKA,Streets and Sanitation,Completed,2018-07-17 07:20:01,2018-08-20 09:49:33,60647,False,True,26.0,41.911968,-87.716579,"(41.91196812978589, -87.71657940338322)",34,2018,2018,7,3,7,POINT (-87.71658 41.91197),349,17,2303,23,Census Tract 2303,23,17031230300,,230300,031


In [0]:
tracts_count = tracts_311.groupby(["namelsad10"]).size().reset_index()
tracts_count = tracts_count.rename(columns={0: "Number_of_Requests"})

In [0]:
tracts_count.sample(n=1)

,namelsad10,Number_of_Requests
403,Census Tract 4908,2844


In [0]:
# load income dataset
income_dir = "/content/gdrive/My Drive/CAPP-30239-Data-Viz/assignment-2/data/income_by_census_tract/income_2018_by_census_tract.csv"
income_df = pd.read_csv(income_dir)
income_df.sample(n=1)

,GEO_ID,NAME,S1901_C01_001E,S1901_C01_001M,S1901_C02_001E,S1901_C02_001M,S1901_C03_001E,S1901_C03_001M,S1901_C04_001E,S1901_C04_001M,S1901_C01_002E,S1901_C01_002M,S1901_C02_002E,S1901_C02_002M,S1901_C03_002E,S1901_C03_002M,S1901_C04_002E,S1901_C04_002M,S1901_C01_003E,S1901_C01_003M,S1901_C02_003E,S1901_C02_003M,S1901_C03_003E,S1901_C03_003M,S1901_C04_003E,S1901_C04_003M,S1901_C01_004E,S1901_C01_004M,S1901_C02_004E,S1901_C02_004M,S1901_C03_004E,S1901_C03_004M,S1901_C04_004E,S1901_C04_004M,S1901_C01_005E,S1901_C01_005M,S1901_C02_005E,S1901_C02_005M,S1901_C03_005E,S1901_C03_005M,...,S1901_C01_012E,S1901_C01_012M,S1901_C02_012E,S1901_C02_012M,S1901_C03_012E,S1901_C03_012M,S1901_C04_012E,S1901_C04_012M,S1901_C01_013E,S1901_C01_013M,S1901_C02_013E,S1901_C02_013M,S1901_C03_013E,S1901_C03_013M,S1901_C04_013E,S1901_C04_013M,S1901_C01_014E,S1901_C01_014M,S1901_C02_014E,S1901_C02_014M,S1901_C03_014E,S1901_C03_014M,S1901_C04_014E,S1901_C04_014M,S1901_C01_015E,S1901_C01_015M,S1901_C02_015E,S1901_C02_015M,S1901_C03_015E,S1901_C03_015M,S1901_C04_015E,S1901_C04_015M,S1901_C01_016E,S1901_C01_016M,S1901_C02_016E,S1901_C02_016M,S1901_C03_016E,S1901_C03_016M,S1901_C04_016E,S1901_C04_016M
503,1400000US17031520300,"Census Tract 5203, Cook County, Illinois",1701,118,1329,130,843,151,372,112,7.5,5.5,8.9,6.2,0.9,1.5,9.4,11.5,4.0,4.5,6.6,6.2,3.8,6.2,0.0,7.1,13.3,5.9,8.6,6.0,6.0,6.3,23.7,12.9,15.6,4.7,12.7,5.0,10.1,7.2,...,41860,6894,47563,13180,65386,11593,32845,9118,56574,11000,52633,6021,N,N,66594,46685,43.0,(X),(X),(X),(X),(X),(X),(X),(X),(X),46.3,(X),(X),(X),(X),(X),(X),(X),(X),(X),(X),(X),31.5,(X)


In [0]:
def extract_census_tract(row):
    return row.split(",")[0]

# Extract Census Tract
income_df["NAME"] = income_df["NAME"].apply(lambda x : extract_census_tract(x))

# Save Estimate!!Households!!Mean income (dollars)
columns = ["NAME", "S1901_C01_013M"]
income_df = save_columns(income_df, columns)
income_df = income_df.rename(columns={"NAME": "namelsad10", "S1901_C01_013M": "Household_Income"})

In [0]:
# Merge it with tracts_count
tracts_count = tracts_count.merge(income_df, on='namelsad10', how='left')
tracts_count = drop_non_numeric(tracts_count, ["Household_Income"], verbose=True)

Removing all non-numeric rows for the following columns: ['Household_Income'].
Removed 3 rows from Household_Income.


In [0]:
tracts_count.sample(n=1)

,namelsad10,Number_of_Requests,Household_Income
648,Census Tract 7306,2961,13193


In [0]:
# Save the merged dataset
tracts_311_count_filename = "/content/gdrive/My Drive/CAPP-30239-Data-Viz/assignment-2/data/tracts_count.csv"
df_to_csv(tracts_count, tracts_311_count_filename, verbose=True)

Successfully saved to /content/gdrive/My Drive/CAPP-30239-Data-Viz/assignment-2/data/tracts_count.csv.


In [0]:
tracts_count

,namelsad10,Number_of_Requests,Household_Income
0,Census Tract 1001,2743,50767
1,Census Tract 1002,3270,10135
2,Census Tract 1003,3354,15620
3,Census Tract 1004,1589,11261
4,Census Tract 1005,3442,11618
...,...,...,...
794,Census Tract 8438,2215,7145
795,Census Tract 8439,2177,5744
796,Census Tract 901,1776,16964
797,Census Tract 902,3189,17667


In [0]:
def evaluateForBivariate(row):
    # Adapted from the slack channel (Credit to Andrew McNutt)
    colorMatrix = [
        ["#64acbe", "#627f8c", "#574249"],
        ["#b0d5df", "#ad9ea5", "#985356"],
        ["#e8e8e8", "#e4acac", "#c85a5a"],
    ]
    xBoundaries = [1165.02, 1959.06]
    xIdx = 0
    if row["Number_of_Requests"] < xBoundaries[0]:
        xIdx = 0
    elif row["Number_of_Requests"] < xBoundaries[1]:
        xIdx = 1
    else:
        xIdx = 2
    yBoundaries = [8200.15, 12498.46]
    yIdx = 0
    if int(row["Household_Income"]) < yBoundaries[0]:
        yIdx = 2
    elif int(row["Household_Income"]) < yBoundaries[1]:
        yIdx = 1
    else:
        yIdx = 0

    return colorMatrix[yIdx][xIdx]

tracts_count["color"] = tracts_count.apply(lambda x : evaluateForBivariate(x), axis = 1)

In [0]:
tracts_count.sample(n=3)

,namelsad10,Number_of_Requests,Household_Income,color
428,Census Tract 513,1133,33236,#64acbe
474,Census Tract 6004,1057,11624,#b0d5df
359,Census Tract 4301.01,1325,14284,#627f8c


In [0]:
# Save the merged dataset
tracts_311_count_filename = "/content/gdrive/My Drive/CAPP-30239-Data-Viz/assignment-2/data/tracts_count.csv"
df_to_csv(tracts_count, tracts_311_count_filename, verbose=True)

Successfully saved to /content/gdrive/My Drive/CAPP-30239-Data-Viz/assignment-2/data/tracts_count.csv.


In [0]:
tracts_311_pothole = tracts_311[(tracts_311['SR_TYPE'] == "Pothole in Street Complaint")]

In [0]:
tracts_311_pothole.sample(n=5)

,SR_NUMBER,SR_TYPE,SR_SHORT_CODE,OWNER_DEPARTMENT,STATUS,CREATED_DATE,CLOSED_DATE,ZIP_CODE,DUPLICATE,LEGACY_RECORD,WARD,LATITUDE,LONGITUDE,LOCATION,DAYS_TAKEN,CREATED_YEAR,CLOSED_YEAR,CREATED_HOUR,CREATED_DAY_OF_WEEK,CREATED_MONTH,geometry,index_right,statefp10,name10,commarea_n,namelsad10,commarea,geoid10,notes,tractce10,countyfp10
2125947,SR19-03067863,Pothole in Street Complaint,PHF,CDOT - Department of Transportation,Completed,2019-12-03 15:35:36,2019-12-09 14:24:47,60652,False,False,18.0,41.736311,-87.723004,"(41.73631053342555, -87.72300395971945)",5,2019,2019,15,3,12,POINT (-87.72300 41.73631),208,17,7003.02,70,Census Tract 7003.02,70,17031700302,,700302,031
1103989,SR18-00143159,Pothole in Street Complaint,PHF,CDOT - Department of Transportation,Completed,2018-07-31 00:14:50,2018-11-13 10:46:13,60656,True,True,41.0,41.981727,-87.805534,"(41.981727, -87.805534)",105,2018,2018,0,3,7,POINT (-87.80553 41.98173),699,17,1006,10,Census Tract 1006,10,17031100600,,100600,031
1862866,SR19-01301877,Pothole in Street Complaint,PHF,CDOT - Department of Transportation,Completed,2019-04-09 20:03:59,2019-04-30 12:25:21,60642,True,False,27.0,41.900844,-87.660149,"(41.90084352161928, -87.66014929153403)",20,2019,2019,20,3,4,POINT (-87.66015 41.90084),113,17,8423,24,Census Tract 8423,24,17031842300,,842300,031
1689902,SR19-01041656,Pothole in Street Complaint,PHF,CDOT - Department of Transportation,Completed,2019-02-20 09:52:09,2019-03-08 13:03:40,60620,False,False,21.0,41.742957,-87.665723,"(41.74295744910078, -87.66572337330922)",16,2019,2019,9,4,2,POINT (-87.66572 41.74296),544,17,7112,71,Census Tract 7112,71,17031711200,,711200,031
1786529,SR19-01190780,Pothole in Street Complaint,PHF,CDOT - Department of Transportation,Completed,2019-03-20 09:43:23,2019-03-20 09:46:18,60640,False,False,40.0,41.978097,-87.669708,"(41.97809745345211, -87.6697079783425)",0,2019,2019,9,4,3,POINT (-87.66971 41.97810),575,17,309,77,Census Tract 309,77,17031030900,,030900,031


In [0]:
# Save DSS dataset
pothole_filename = "/content/gdrive/My Drive/CAPP-30239-Data-Viz/assignment-2/data/tracts_311_pothole.csv"
df_to_csv(tracts_311_pothole, pothole_filename, verbose=True)

Successfully saved to /content/gdrive/My Drive/CAPP-30239-Data-Viz/assignment-2/data/tracts_311_pothole.csv.


In [0]:
tracts_311_pothole.shape

(110816, 31)

In [0]:
tracts_311_call = tracts_311[(tracts_311['SR_TYPE'] == "311 INFORMATION ONLY CALL")]

# Save 311 call dataset
call_filename = "/content/gdrive/My Drive/CAPP-30239-Data-Viz/assignment-2/data/tracts_311_call.csv"
df_to_csv(tracts_311_call, call_filename, verbose=True)

Successfully saved to /content/gdrive/My Drive/CAPP-30239-Data-Viz/assignment-2/data/tracts_311_call.csv.


In [0]:
tracts_311_call.sample(n=5)

,SR_NUMBER,SR_TYPE,SR_SHORT_CODE,OWNER_DEPARTMENT,STATUS,CREATED_DATE,CLOSED_DATE,ZIP_CODE,DUPLICATE,LEGACY_RECORD,WARD,LATITUDE,LONGITUDE,LOCATION,DAYS_TAKEN,CREATED_YEAR,CLOSED_YEAR,CREATED_HOUR,CREATED_DAY_OF_WEEK,CREATED_MONTH,geometry,index_right,statefp10,name10,commarea_n,namelsad10,commarea,geoid10,notes,tractce10,countyfp10
2103696,SR19-03017394,311 INFORMATION ONLY CALL,311IOC,311 City Services,Completed,2019-11-25 10:28:45,2019-11-25 10:28:45,60612,False,False,28.0,41.871831,-87.679846,"(41.871831277993564, -87.67984621876099)",0,2019,2019,10,2,11,POINT (-87.67985 41.87183),5,17,8382,28,Census Tract 8382,28,17031838200,,838200,031
82154,SR19-01881304,311 INFORMATION ONLY CALL,311IOC,311 City Services,Completed,2019-07-07 18:46:55,2019-07-07 18:46:55,60612,False,False,28.0,41.871831,-87.679846,"(41.871831277993564, -87.67984621876099)",0,2019,2019,18,1,7,POINT (-87.67985 41.87183),5,17,8382,28,Census Tract 8382,28,17031838200,,838200,031
1752729,SR19-01137806,311 INFORMATION ONLY CALL,311IOC,311 City Services,Completed,2019-03-11 11:07:31,2019-03-11 11:07:31,60612,False,False,28.0,41.871831,-87.679846,"(41.87183127800069, -87.67984622001485)",0,2019,2019,11,2,3,POINT (-87.67985 41.87183),5,17,8382,28,Census Tract 8382,28,17031838200,,838200,031
2351355,SR20-03320440,311 INFORMATION ONLY CALL,311IOC,311 City Services,Completed,2020-01-14 10:18:00,2020-01-14 10:18:01,60612,False,False,28.0,41.871831,-87.679846,"(41.871831277993564, -87.67984621876099)",0,2020,2020,10,3,1,POINT (-87.67985 41.87183),5,17,8382,28,Census Tract 8382,28,17031838200,,838200,031
874022,SR19-02699102,311 INFORMATION ONLY CALL,311IOC,311 City Services,Completed,2019-10-10 17:09:17,2019-10-10 17:09:17,60612,False,False,28.0,41.871831,-87.679846,"(41.871831277993564, -87.67984621876099)",0,2019,2019,17,5,10,POINT (-87.67985 41.87183),5,17,8382,28,Census Tract 8382,28,17031838200,,838200,031
